In [31]:
import numpy as np # for linear algebra
import pandas as pd # data processing, CSV file I/O, etc
import seaborn as sns # for plots
import plotly.graph_objects as go # for plots
import plotly.express as px #for plots
import matplotlib.pyplot as plt # for visualizations and plots

# this eliminates the requirement to use plt.show() after every plot
%matplotlib inline

# changing the default figure sizes
from pylab import rcParams
rcParams['figure.figsize'] = 15, 10

import random # random library
pallete = ['Accent_r', 'Blues', 'BrBG', 'BrBG_r', 'BuPu', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'OrRd', 'Oranges', 'Paired', 'PuBu', 'PuBuGn', 'PuRd', 'Purples', 'RdGy_r', 'RdPu', 'Reds', 'autumn', 'cool', 'coolwarm', 'flag', 'flare', 'gist_rainbow', 'hot', 'magma', 'mako', 'plasma', 'prism', 'rainbow', 'rocket', 'seismic', 'spring', 'summer', 'terrain', 'turbo', 'twilight']

import os

In [32]:
df = pd.read_csv("raw/world_diabetes.csv")
df.head() # displays the top 5 values in the dataset

,Entity,Code,Year,Diabetes prevalence (% of population ages 20 to 79)
0,Afghanistan,AFG,2011,7.6
1,Afghanistan,AFG,2024,11.7
2,Albania,ALB,2011,2.8
3,Albania,ALB,2024,10.6
4,Algeria,DZA,2011,7.0


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455 entries, 0 to 454
Data columns (total 4 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Entity                                               455 non-null    object 
 1   Code                                                 431 non-null    object 
 2   Year                                                 455 non-null    int64  
 3   Diabetes prevalence (% of population ages 20 to 79)  455 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 14.3+ KB


In [34]:
df.describe()

,Year,Diabetes prevalence (% of population ages 20 to 79)
count,455.000000,455.000000
mean,2016.931868,9.173607
std,7.204515,4.962085
min,2000.000000,0.000000
25%,2011.000000,5.650000
50%,2011.000000,8.100000
75%,2024.000000,11.304949
max,2024.000000,31.400000


In [35]:
df.isnull().sum()

Entity                                                  0
Code                                                   24
Year                                                    0
Diabetes prevalence (% of population ages 20 to 79)     0
dtype: int64

## Data Cleaning

In [36]:
# cleanup regions
df_cleaned = df.dropna() # Drop the row which missing value which is country code since they are regions
df_cleaned = df_cleaned[df_cleaned['Code'] != 'OWID_WRL'] # clean world entries
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 429 entries, 0 to 454
Data columns (total 4 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Entity                                               429 non-null    object 
 1   Code                                                 429 non-null    object 
 2   Year                                                 429 non-null    int64  
 3   Diabetes prevalence (% of population ages 20 to 79)  429 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 16.8+ KB


In [37]:
df_cleaned.describe()

,Year,Diabetes prevalence (% of population ages 20 to 79)
count,429.000000,429.000000
mean,2016.897436,9.167832
std,7.243671,5.055435
min,2000.000000,0.000000
25%,2011.000000,5.500000
50%,2011.000000,8.000000
75%,2024.000000,11.500000
max,2024.000000,31.400000


In [38]:
df_cleaned.head()

,Entity,Code,Year,Diabetes prevalence (% of population ages 20 to 79)
0,Afghanistan,AFG,2011,7.6
1,Afghanistan,AFG,2024,11.7
2,Albania,ALB,2011,2.8
3,Albania,ALB,2024,10.6
4,Algeria,DZA,2011,7.0


### Pivoted the dataset

In [39]:
# Check if required columns exist
required_cols = ['Entity', 'Code', 'Year']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"Error: Missing required columns: {', '.join(missing_cols)}")
    print(f"Available columns: {', '.join(df.columns)}")
    sys.exit(1)

# Get the value column name (the 4th column, assuming it's the data column)
value_cols = [col for col in df_cleaned.columns if col not in ['Entity', 'Code', 'Year']]

if not value_cols:
    print("Error: No value column found to pivot.")
    sys.exit(1)

value_col = value_cols[0]
print(f"Pivoting column: '{value_col}'")

# Pivot the data
df_pivoted = df_cleaned.pivot_table(
    index=['Entity', 'Code'],
    columns='Year',
    values=value_col,
    aggfunc='first'  # Use 'first' in case of duplicates
)

# Reset index to make Entity and Code regular columns
df_pivoted = df_pivoted.reset_index()

# Rename columns to include original column name for clarity
df_pivoted.columns.name = None  # Remove the 'Year' label from column index
year_cols = [col for col in df_pivoted.columns if col not in ['Entity', 'Code']]

# Sort columns: Entity, Code, then years in ascending order
sorted_cols = ['Entity', 'Code'] + sorted(year_cols)
df_pivoted = df_pivoted[sorted_cols]

df_pivoted.head()

Pivoting column: 'Diabetes prevalence (% of population ages 20 to 79)'


,Entity,Code,2000,2011,2024
0,Afghanistan,AFG,NaN,7.6,11.7
1,Albania,ALB,NaN,2.8,10.6
2,Algeria,DZA,NaN,7.0,17.5
3,Andorra,AND,NaN,5.4,10.1
4,Angola,AGO,NaN,2.9,4.9


In [40]:
df_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Entity  209 non-null    object 
 1   Code    209 non-null    object 
 2   2000    17 non-null     float64
 3   2011    203 non-null    float64
 4   2024    209 non-null    float64
dtypes: float64(3), object(2)
memory usage: 8.3+ KB


In [41]:
df_pivoted.describe()

,2000,2011,2024
count,17.000000,203.000000,209.000000
mean,11.435294,8.027094,10.091388
std,4.491651,4.333956,5.499888
min,0.000000,1.900000,1.500000
25%,11.800000,5.100000,6.500000
50%,12.100000,7.500000,8.500000
75%,14.100000,9.550000,12.200000
max,15.500000,25.300000,31.400000


### Drop NaN

In [42]:
# Drop 2000 column since they missing most of data
df_dropped = df_pivoted.drop(columns=[int(2000)])

# drop some countries with missing 2011 values
df_dropped = df_dropped.dropna()

df_dropped.head()
# print(df_dropped.head())

,Entity,Code,2011,2024
0,Afghanistan,AFG,7.6,11.7
1,Albania,ALB,2.8,10.6
2,Algeria,DZA,7.0,17.5
3,Andorra,AND,5.4,10.1
4,Angola,AGO,2.9,4.9


In [43]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Index: 203 entries, 0 to 208
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Entity  203 non-null    object 
 1   Code    203 non-null    object 
 2   2011    203 non-null    float64
 3   2024    203 non-null    float64
dtypes: float64(2), object(2)
memory usage: 7.9+ KB


In [44]:
df_dropped.describe()

,2011,2024
count,203.000000,203.000000
mean,8.027094,10.165517
std,4.333956,5.533972
min,1.900000,1.500000
25%,5.100000,6.600000
50%,7.500000,8.500000
75%,9.550000,12.250000
max,25.300000,31.400000


## Features Engineering

In [45]:
# Add calculated columns
df_dropped['Prevalence_Change'] = df_dropped[int('2024')] - df_dropped[int('2011')]
df_dropped['Percent_Change'] = ((df_dropped[int('2024')] - df_dropped[int('2011')]) / df_dropped[int('2011')]) * 100

# Sort by biggest changes
df_global_sorted = df_dropped.sort_values('Prevalence_Change', ascending=False)
df_global_sorted.head()

,Entity,Code,2011,2024,Prevalence_Change,Percent_Change
144,Pakistan,PAK,7.9,31.4,23.5,297.468354
162,Samoa,WSM,7.7,25.4,17.7,229.870130
67,French Polynesia,PYF,8.6,22.8,14.2,165.116279
135,New Caledonia,NCL,8.6,22.0,13.4,155.813953
76,Guam,GUM,8.6,20.3,11.7,136.046512


In [46]:
df_global_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 203 entries, 144 to 208
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Entity             203 non-null    object 
 1   Code               203 non-null    object 
 2   2011               203 non-null    float64
 3   2024               203 non-null    float64
 4   Prevalence_Change  203 non-null    float64
 5   Percent_Change     203 non-null    float64
dtypes: float64(4), object(2)
memory usage: 11.1+ KB


In [47]:
df_global_sorted.describe()

,2011,2024,Prevalence_Change,Percent_Change
count,203.000000,203.000000,203.000000,203.000000
mean,8.027094,10.165517,2.138424,39.463918
std,4.333956,5.533972,3.969887,65.793148
min,1.900000,1.500000,-8.300000,-84.693878
25%,5.100000,6.600000,-0.200000,-2.317466
50%,7.500000,8.500000,1.600000,23.671498
75%,9.550000,12.250000,3.900000,56.613191
max,25.300000,31.400000,23.500000,305.000000


In [49]:
df_global_sorted.to_csv('new/world.csv', index=False)